# Committee 업무 중 The most time-consuming task인 문서 취합 효율화 플랫폼
- 기능 1 : 여러 Word 파일들을 한 화면에서 동시 편집 가능
- 기능 2 : 일부 수정된 Word 파일을 Local PC에 저장 혹은 작성 부서에 e-mail 전송
- 기능 3 : 최종 수정된 Word 파일들을 pdf로 통합
- 기능 4 : LLM을 활용하여, 오타 및 논리적 오류가 보이는 부분 Early-alert

In [5]:
!pip install streamlit
!pip install pyngrok
!pip install python-docx PyPDF2
!pip install langchain_community
!pip install unstructured
!pip install nltk
!pip install python-docx
!pip install langchain langchain-openai openai
!pip install docx2pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1

In [6]:
!sudo apt-get update
!sudo apt-get install -y libreoffice
!sudo apt-get install fonts-nanum

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,573 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,411 kB]
Get:14 ht

In [1]:
%%writefile read_docx_util.py
import nltk
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

nltk.download('punkt')

def read_docx(file):
  loader = UnstructuredWordDocumentLoader(
      file,
      mode='elements', # 요소 단위 분할
      strategy='hi_res' # 표 구조 인식 활성화
  )
  raw_docs = loader.load()
  grouped_sections = []
  current_section_title = "Introduction"
  current_text_buffer = []

  for doc in raw_docs:
    category = doc.metadata.get("category")
    content = doc.page_content.strip()

    emphasized_contents = doc.metadata.get("emphasized_text_contents",[])
    emphasized_tags = doc.metadata.get("emphasized_text_tags", [])
    is_real_title = False

    if 'b' in emphasized_tags:
      clean_emphasized=' '.join([t.strip() for t in emphasized_contents])
      if content == clean_emphasized:
        is_real_title = True

    if category == "Table":
      html_content = doc.metadata.get("text_as_html","")
      if not html_content:
        html_content = f"<div>{content}</div>"
      formatted_table = f"\n[Table Start]\n{html_content}\n[Table End]\n"
      current_text_buffer.append(formatted_table)
    elif is_real_title:
      # If there's content in the buffer, it belongs to the previous section
      if current_text_buffer:
        grouped_sections.append({
            "title": current_section_title,
            "content": "\n".join(current_text_buffer)
        })
      # Start a new section with the new title
      current_section_title = content # Update the section title
      current_text_buffer = [] # Reset the buffer for the new section
    else:
      current_text_buffer.append(content)

  # After the loop, append any remaining content as the last section
  if current_text_buffer:
    grouped_sections.append({
            "title": current_section_title,
            "content": "\n".join(current_text_buffer)
        })
  return grouped_sections


Writing read_docx_util.py


### 아래의 코드는 Window환경에서만 작동하므로, 추후에 실제 window에서 실행할 때에 사용하자
### 임시로 리눅스 환경에서 작동이 되는 pdf_converter.py를 생성하였으니, PoC 단계에서는 이를 활용할 것


In [ ]:
# @title
import os
from docx2pdf import convert
from pypdf import PdfWriter
import win32com
from win32com.client import gencache
import shutil

ppSaveAsPdf=32

def get_files(folder_path):
  extentions = ('.docx', 'doc', 'pptx', 'ppt')
  files = [f for f in os.listdir(folder_path) if f.lower().endswith(extentions) and not f.startswith('~$')]
  return sorted(files)

def convert_ppt_to_pdf(input_path, output_path):
  powerpoint = None
  presentation = None
  try:
    powerpoint = win32com.client.Dispatch("Powerpoint.Application")
    presentation = powerpoint.Presentations.Open(input_path, WithWindow=False)
    presentation.SaveAs(output_path, FileFormat=ppSaveAsPdf)
    return True
  except Exception as e:
    print(f"ppt 변환 에러 : {e}")
    return False
  finally:
    if presentation:
      presentation.Close()
    if powerpoint:
      try:
        if powerpoint.Presentations.Count ==0:
          powerpoint.Quit()
      except:
        pass

def convert_to_pdf_wrapper(input_path, output_path):
  ext = os.path.splitext(input_path)[1].lower()
  if ext in ['.docx', '.doc']:
    convert(input_path, output_path)
  elif ext in ['.pptx', '.ppt']:
    success = convert_ppt_to_pdf(input_path, output_path)
    if not success:
      raise Exception("PowerPoint conversion failed via win32com")
  else:
    raise Exception(f"지원하지 않는 확장자입니다. 확장자 : {ext}")

def convert_each_to_pdf(folder_path, output_folder=None):
  if output_folder is None:
    output_folder = folder_path
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  target_files = get_files(folder_path)

  print(f"총 {len(target_files)}개의 파일을 개별 PDF로 변환합니다.")

  for file in target_files:
    input_path = os.path.join(folder_path, file)
    output_filename = os.path.splitext(file)[0] + ".pdf"
    output_path = os.path.join(output_folder, output_filename)
    try:
      convert_to_pdf_wrapper(input_path, output_path)
      print(f"[완료]{file} -> {output_filename}")
    except Exception as e:
      print(f"{file} PDF 변환 실패 : {e}")

def cleanup_gen_py():
  """win32com gen_py 캐시 폴더를 자동으로 삭제."""
  paths = set()
  gen_attr = getattr(win32com, "__gen_path__", None)
  if gen_attr:
    paths.add(gen_attr)
    try:
      gen_path = gencache.GetGeneratePath()
      if gen_path:
        paths.add(gen_path)
    except Exception:
      pass
  for p in paths:
    if p and os.path.isdir(p):
      try:
        print(f"[cleanup_gen_py] 삭제 시도: {p}")
        shutil.rmtree(p, ignore_errors=True)
      except Exception as e:
        print(f"[cleanup_gen_py] 삭제 실패: {p} -> {e}")


ModuleNotFoundError: No module named 'win32com'

In [2]:
%%writefile pdf_converter.py
import os
import subprocess
import sys

def convert_to_pdf_linux(input_path, output_folder):
    """
    LibreOffice를 사용하여 단일 파일을 PDF로 변환합니다.
    """
    try:
        # libreoffice 명령어가 설치되어 있는지 확인이 필요하지만,
        # 여기서는 설치되었다고 가정하고 실행합니다.
        cmd = [
            'libreoffice',
            '--headless',
            '--convert-to', 'pdf',
            input_path,
            '--outdir', output_folder
        ]

        # 명령어 실행 (타임아웃 60초 설정)
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=60)

        if result.returncode != 0:
            return False, result.stderr.decode('utf-8')

        return True, "Success"
    except subprocess.TimeoutExpired:
        return False, "Timeout"
    except Exception as e:
        return False, str(e)

def batch_convert_to_pdf(target_folder):
    """
    지정된 폴더 내의 Word, PPT 파일을 모두 찾아 PDF로 변환합니다.
    """
    if not os.path.exists(target_folder):
        yield "Error", f"폴더를 찾을 수 없습니다: {target_folder}"
        return

    # 결과가 저장될 폴더 (원본폴더/pdf_output)
    output_folder = os.path.join(target_folder, "pdf_output")
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extensions = ('.docx', '.doc', '.pptx', '.ppt')
    files = [f for f in os.listdir(target_folder) if f.lower().endswith(extensions) and not f.startswith('~$')]

    if not files:
        yield "Info", "변환할 지원 파일(.docx, .pptx 등)이 없습니다."
        return

    yield "Info", f"총 {len(files)}개의 파일을 변환합니다. 결과 저장 경로: {output_folder}"

    for file in files:
        input_path = os.path.join(target_folder, file)
        yield "Progress", f"변환 중: {file}..."

        success, msg = convert_to_pdf_linux(input_path, output_folder)

        if success:
            yield "Success", f"[성공] {file}"
        else:
            yield "Error", f"[실패] {file} : {msg}"

Writing pdf_converter.py


In [7]:
%%writefile highlighting.py
import json
import re

def highlight_errors(original_text, analysis_result_json):
    """
    원본 텍스트에서 error_sentence를 찾아 빨간색 배경 처리를 합니다.
    """
    highlighted_text = original_text

    try:
        # 1. JSON 전처리
        clean_json = analysis_result_json.replace("```json", "").replace("```", "").strip()
        errors = json.loads(clean_json)

        if not isinstance(errors, list):
            return original_text, []

        for error in errors:
            target = error.get("error_sentence", "").strip()

            if not target:
                continue

            # 2. 단순 replace 시도
            if target in highlighted_text:
                replacement = f"<span style='background-color: #ffdce0; color: #d8000c; font-weight: bold; padding: 2px 4px; border-radius: 4px;'>{target}</span>"
                highlighted_text = highlighted_text.replace(target, replacement)

            # 3. [보완] 단순 매칭 실패 시, 공백/줄바꿈을 유연하게 처리하여 검색
            else:
                # target의 특수문자를 이스케이프하고, 공백을 유연한 정규식 패턴(\s+)으로 변경
                # 예: "안녕 하세요" -> "안녕\s+하세요" (줄바꿈이나 여러 공백도 매칭됨)
                import re
                escaped_target = re.escape(target)
                pattern = escaped_target.replace(r"\ ", r"\s+")

                # HTML 태그로 감싸기 위한 정규식 치환
                replacement = f"<span style='background-color: #ffdce0; color: #d8000c; font-weight: bold; padding: 2px 4px; border-radius: 4px;'>\g<0></span>"

                # 원본 텍스트가 이미 HTML 태그 등으로 오염되지 않았다고 가정하고 수행
                highlighted_text = re.sub(pattern, replacement, highlighted_text)

        return highlighted_text, errors

    except json.JSONDecodeError:
        return original_text, []
    except Exception:
        return original_text, []

Overwriting highlighting.py


### 아래는 LLM 추가 전 app.py 코드

In [ ]:
# @title
%%writefile app.py
import streamlit as st
import os
import tempfile
import docx  # [추가] 원본 docx를 단순하게 읽기 위해 필요
from read_docx_util import read_docx # 복잡한 로직(Unstructured)은 여기서 import (나중에 결과창 등에 사용)
from PyPDF2 import PdfReader

# --- [신규 함수] 원본 내용을 단순하게 보여주기 위한 함수 ---
def read_raw_docx(file_path):
    """
    복잡한 로직 없이 docx 파일의 문단(Paragraph) 텍스트를 그대로 읽어옵니다.
    """
    try:
        doc = docx.Document(file_path)
        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)
        return '\n'.join(full_text)
    except Exception as e:
        return f"파일을 읽는 중 오류가 발생했습니다: {str(e)}"

def read_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def main():
    st.set_page_config(page_title="Word 취합 및 교정, Pdf 통합 플랫폼", layout='wide')

    if "analysis_cache" not in st.session_state:
        st.session_state.analysis_cache = {}

    st.title("Word 문서 취합/교정/통합 PDF 생성 tool")

    col1, col2 = st.columns([1, 1])

    with col1:
        st.header("원본 문서 (Raw View)")
        original_content_placeholder = st.empty()

    with col2:
        st.header("결과 (PDF 변환 / 오류 검수)")
        results_content_placeholder = st.empty()

    st.sidebar.header("설정")
    uploaded_file = st.sidebar.file_uploader("Word 또는 PDF 파일을 업로드하세요", type=["docx", "pdf"])

    # (참고) 실제 파일 처리를 위한 경로가 필요할 수 있으므로 저장
    output_pdf = st.sidebar.text_input("최종 통합 pdf 저장 경로", value=os.path.join(os.getcwd(), 'Integrated.pdf'))

    if uploaded_file is not None:
        file_details = {"FileName": uploaded_file.name, "FileType": uploaded_file.type, "FileSize": uploaded_file.size}
        st.sidebar.write(file_details)

        # 왼쪽 화면(원본 문서) 구성
        with original_content_placeholder.container():
            st.subheader(f"업로드된 파일: {uploaded_file.name}")

            if uploaded_file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
                # 임시 파일 생성
                with tempfile.NamedTemporaryFile(delete=False, suffix='.docx') as tmp_file:
                    tmp_file.write(uploaded_file.getvalue())
                    tmp_file_path = tmp_file.name

                try:
                    # --- [수정됨] 왼쪽 화면에는 새로 만든 단순 읽기 함수 연결 ---
                    raw_content = read_raw_docx(tmp_file_path)
                    st.text_area("내용 확인", raw_content, height=600)

                    # (참고) 만약 오른쪽 화면이나 백그라운드 처리를 위해
                    # 복잡한 로직(read_docx)이 필요하다면 여기서 호출하거나 세션에 저장합니다.
                    # processed_data = read_docx(tmp_file_path)

                except Exception as e:
                    st.error(f"오류 발생: {e}")
                finally:
                    # 임시 파일 삭제
                    if os.path.exists(tmp_file_path):
                        os.remove(tmp_file_path)

            elif uploaded_file.type == "application/pdf":
                pdf_content = read_pdf(uploaded_file)
                st.text_area("원본 PDF 문서 내용", pdf_content, height=600)
            else:
                st.warning("지원되지 않는 파일 형식입니다.")

        with results_content_placeholder.container():
            st.info("파일이 업로드되었습니다. 분석 버튼을 누르거나 추가 작업을 진행하세요.")

    st.sidebar.markdown("---")

if __name__ == "__main__":
    main()

Overwriting app.py


### 아래는 LLM 추가 후 app.py 코드

In [ ]:
# @title
%%writefile app.py
import streamlit as st
import os
import tempfile
import docx
from read_docx_util import read_docx
from PyPDF2 import PdfReader

# --- LangChain 관련 임포트 ---
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 단순 읽기 함수 (왼쪽 화면용) ---
def read_raw_docx(file_path):
    try:
        doc = docx.Document(file_path)
        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)
        return '\n'.join(full_text)
    except Exception as e:
        return f"파일을 읽는 중 오류가 발생했습니다: {str(e)}"

def read_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# --- 오타 검수 체인 생성 함수 ---
def get_proofreading_chain(api_key):
    # LLM 모델 설정 (gpt-4o 또는 gpt-3.5-turbo 등 사용)
    llm = ChatOpenAI(
        model="gpt-5-mini",
        temperature=0,
        openai_api_key=api_key
    )

    # 프롬프트 템플릿 작성
    template = """
    당신은 전문적인 한국어 교정 교열자입니다.
    아래 제공된 텍스트에서 오타, 맞춤법 오류, 어색한 문장 표현을 찾아내어 교정해주세요.

    [분석할 텍스트]:
    {text}

    [요청 사항]:
    1. 오타나 오류가 있는 부분만 구체적으로 지적하고 수정안을 제시하세요.
    2. 오류가 없다면 "수정 사항 없음"이라고만 출력하세요.
    3. 결과는 읽기 편하게 마크다운 리스트 형태로 출력하세요.
    """

    prompt = PromptTemplate.from_template(template)

    # 체인 연결: Prompt -> LLM -> String Output
    return prompt | llm | StrOutputParser()

def main():
    st.set_page_config(page_title="Word 취합 및 교정 플랫폼", layout='wide')

    # 세션 상태 초기화
    if "proofreading_results" not in st.session_state:
        st.session_state.proofreading_results = None

    st.title("Word 문서 취합/교정/통합 PDF 생성 tool")

    # --- 사이드바 설정 ---
    st.sidebar.header("설정")

    # API 키 입력 받기 (보안을 위해 비밀번호 타입으로 설정)
    openai_api_key = st.sidebar.text_input("OpenAI API Key", type="password")

    uploaded_file = st.sidebar.file_uploader("Word 또는 PDF 파일을 업로드하세요", type=["docx", "pdf"])

    st.sidebar.markdown("---")

    # --- 메인 레이아웃 ---
    col1, col2 = st.columns([1, 1])

    with col1:
        st.header("📄 원본 문서 (Raw View)")
        original_content_placeholder = st.empty()

    with col2:
        st.header("🔍 오타 검수 및 결과")
        # 검수 시작 버튼 위치
        action_placeholder = st.empty()
        # 결과 보여줄 위치
        results_placeholder = st.container()

    if uploaded_file is not None:
        # 임시 파일 저장 (공통 로직)
        suffix = '.docx' if uploaded_file.name.endswith('.docx') else '.pdf'
        with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp_file:
            tmp_file.write(uploaded_file.getvalue())
            tmp_file_path = tmp_file.name

        try:
            # 1. 왼쪽 화면: 원본 단순 텍스트 표시
            with original_content_placeholder.container():
                st.subheader(f"파일명: {uploaded_file.name}")
                if suffix == '.docx':
                    raw_content = read_raw_docx(tmp_file_path)
                    st.text_area("내용 확인", raw_content, height=600)
                elif suffix == '.pdf':
                    pdf_content = read_pdf(uploaded_file)
                    st.text_area("내용 확인", pdf_content, height=600)

            # 2. 오른쪽 화면: 오타 검수 기능
            with action_placeholder.container():
                # Word 파일일 경우에만 구조화된 분석 가능
                if suffix == '.docx':
                    if st.button("오타 검수 시작 (AI)", type="primary"):
                        if not openai_api_key:
                            st.error("사이드바에 OpenAI API Key를 입력해주세요!")
                        else:
                            # 1) 문서 구조화 로딩
                            sections = read_docx(tmp_file_path)

                            # 2) LLM 체인 준비
                            chain = get_proofreading_chain(openai_api_key)

                            results = []
                            progress_bar = st.progress(0)
                            status_text = st.empty()

                            # 3) 섹션별 반복 검수
                            for i, section in enumerate(sections):
                                title = section['title']
                                content = section['content']

                                status_text.text(f"분석 중... ({i+1}/{len(sections)}): {title}")

                                # LLM 호출
                                try:
                                    response = chain.invoke({"text": content})
                                    results.append({
                                        "title": title,
                                        "original": content,
                                        "correction": response
                                    })
                                except Exception as e:
                                    results.append({
                                        "title": title,
                                        "correction": f"에러 발생: {str(e)}"
                                    })

                                # 진행률 업데이트
                                progress_bar.progress((i + 1) / len(sections))

                            # 4) 결과 세션에 저장
                            st.session_state.proofreading_results = results
                            status_text.text("분석 완료!")
                            progress_bar.empty()

        finally:
            # 임시 파일 삭제
            if os.path.exists(tmp_file_path):
                os.remove(tmp_file_path)

        # 3. 결과 출력 (세션 상태에 값이 있으면 표시)
        if st.session_state.proofreading_results:
            with results_placeholder:
                st.success("검수가 완료되었습니다.")
                for res in st.session_state.proofreading_results:
                    with st.expander(f"📌 {res['title']}", expanded=True):
                        st.markdown("**[교정 제안]**")
                        st.markdown(res['correction'])
                        st.markdown("---")
                        st.caption("**[원본 텍스트]**")
                        st.text(res.get('original', '')[:200] + "...") # 너무 길면 자르기

if __name__ == "__main__":
    main()

Overwriting app.py


### 아래는 LLM에 PDF 변환 기능까지 추가한 버전

In [ ]:
%%writefile app.py
import streamlit as st
import os
import tempfile
import docx
from read_docx_util import read_docx
from pdf_converter import batch_convert_to_pdf  # [추가] 방금 만든 변환 모듈 임포트
from PyPDF2 import PdfReader

# --- LangChain 관련 임포트 (기존 유지) ---
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 기존 함수들 (read_raw_docx, read_pdf, get_proofreading_chain) ---
def read_raw_docx(file_path):
    try:
        doc = docx.Document(file_path)
        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)
        return '\n'.join(full_text)
    except Exception as e:
        return f"파일을 읽는 중 오류가 발생했습니다: {str(e)}"

def read_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def get_proofreading_chain(api_key):
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0, openai_api_key=api_key)
    template = """
    당신은 전문적인 한국어 교정 교열자입니다.
    아래 텍스트에서 오타, 맞춤법 오류, 어색한 문장 표현을 찾아내어 교정해주세요.

    [텍스트]: {text}

    [요청]: 오류가 있는 부분만 구체적으로 지적하고 수정안을 제시하세요. 없으면 "수정 사항 없음".
    """
    prompt = PromptTemplate.from_template(template)
    return prompt | llm | StrOutputParser()

# --- 메인 함수 ---
def main():
    st.set_page_config(page_title="Word/PPT 통합 플랫폼", layout='wide')

    if "proofreading_results" not in st.session_state:
        st.session_state.proofreading_results = None

    st.title("문서 작업 통합 도구 (Correction & Conversion)")

    # [UI 구조 변경] 탭을 사용하여 기능 분리
    tab1, tab2 = st.tabs(["📄 문서 오타 검수", "🔄 PDF 일괄 변환"])

    # =========================================================
    # TAB 1: 기존 오타 검수 기능
    # =========================================================
    with tab1:
        col1, col2 = st.columns([1, 1])

        with col1:
            st.subheader("1. 파일 업로드 및 확인")
            openai_api_key = st.text_input("OpenAI API Key", type="password", key="api_key_tab1")
            uploaded_file = st.file_uploader("검수할 파일 업로드 (Word/PDF)", type=["docx", "pdf"], key="uploader_tab1")

            # [수정] 오타 검수 로직
            if uploaded_file and suffix == '.docx':
                if st.button("오타 검수 시작", type="primary"):
                    st.info("문서 분석 및 AI 검수를 시작합니다...")

                    # 1) 문서 구조 읽기
                    sections = read_docx(tmp_file_path)
                    chain = get_proofreading_chain(openai_api_key)

                    # 결과 저장용
                    final_results = []

                    # 진행률 표시
                    progress_text = "검수 진행 중..."
                    my_bar = st.progress(0, text=progress_text)

                    # 전체 하이라이트된 텍스트를 모을 변수
                    full_highlighted_html = ""

                    for i, section in enumerate(sections):
                        title = section.get('title', '제목 없음')
                        content = section.get('content', '')

                        # LLM 호출
                        try:
                            response = chain.invoke({"text": content})

                            # [핵심] 하이라이팅 처리
                            hl_text, errors_list = highlight_errors(content, response)

                            # 결과 데이터 구조화
                            final_results.append({
                                "title": title,
                                "original_content": content,
                                "highlighted_content": hl_text, # 하이라이트된 HTML
                                "errors": errors_list
                            })

                            # 전체 뷰를 위해 섹션별 HTML 누적
                            full_highlighted_html += f"<h3>{title}</h3>{hl_text}<hr>"

                        except Exception as e:
                            st.error(f"에러 발생: {e}")

                        my_bar.progress((i + 1) / len(sections), text=f"{progress_text} ({i+1}/{len(sections)})")

                    my_bar.empty()

                    # 세션에 저장
                    st.session_state.proofreading_results = final_results
                    st.session_state.full_highlighted_view = full_highlighted_html

            # [화면 갱신 로직]
            # 검수가 완료되어 데이터가 있다면 -> 왼쪽 화면을 하이라이트 뷰로 교체
            if "full_highlighted_view" in st.session_state and st.session_state.full_highlighted_view:
                with col1:
                    st.subheader("🔴 오타 식별 결과 (원본)")
                    # unsafe_allow_html=True를 해야 HTML 태그가 먹힙니다.
                    st.markdown(
                        f"<div style='height: 600px; overflow-y: auto; border: 1px solid #ccc; padding: 10px; border-radius: 5px;'>{st.session_state.full_highlighted_view}</div>",
                        unsafe_allow_html=True
                    )

            # [오른쪽 화면] 수정 제안 목록 표시
            if st.session_state.proofreading_results:
                with st.container(): # col2가 아니라 결과창 위치에 표시
                    st.subheader("💡 수정 제안")
                    for res in st.session_state.proofreading_results:
                        errors = res.get("errors", [])
                        if errors:
                            with st.expander(f"📌 {res['title']} ({len(errors)}건 발견)", expanded=True):
                                for err in errors:
                                    st.markdown(f"**❌ [오류]**: {err.get('error_sentence')}")
                                    st.markdown(f"**✅ [수정]**: {err.get('correction')}")
                                    st.caption(f"Reason: {err.get('reason')}")
                                    st.divider()
                        else:
                            with st.expander(f"✅ {res['title']} - 오류 없음", expanded=False):
                                st.write("발견된 오류가 없습니다.")

    # =========================================================
    # TAB 2: [신규] PDF 일괄 변환 기능
    # =========================================================
    with tab2:
        st.header("📂 Word/PPT -> PDF 일괄 변환")
        st.markdown("""
        - 지정한 폴더에 있는 **모든 .docx, .pptx 파일**을 PDF로 변환합니다.
        - **주의:** 서버(Linux)에 LibreOffice와 한글 폰트가 설치되어 있어야 정상 동작합니다.
        """)

        # 기본값을 현재 작업 경로로 설정
        default_path = os.getcwd()
        target_folder = st.text_input("변환할 파일이 있는 폴더 경로를 입력하세요:", value=default_path)

        if st.button("일괄 변환 시작", type="primary"):
            st.write("---")
            log_area = st.empty()

            # 제너레이터(yield)로부터 메시지를 받아 실시간 출력
            for msg_type, msg in batch_convert_to_pdf(target_folder):
                if msg_type == "Error":
                    st.error(msg)
                elif msg_type == "Success":
                    st.success(msg)
                elif msg_type == "Info":
                    st.info(msg)
                elif msg_type == "Progress":
                    with log_area:
                        st.write(f"⏳ {msg}")

            st.success("모든 작업이 종료되었습니다.")
            log_area.empty()

if __name__ == "__main__":
    main()

Overwriting app.py


### 아래는 하이라이팅 기능까지 추가된 버전

In [22]:
%%writefile app.py
import streamlit as st
import os
import tempfile
import docx
from read_docx_util import read_docx
from pdf_converter import batch_convert_to_pdf  # [추가] 방금 만든 변환 모듈 임포트
from PyPDF2 import PdfReader
from highlighting import highlight_errors

# --- LangChain 관련 임포트 (기존 유지) ---
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import re  # [필수] 이 줄이 없으면 에러가 납니다.

# [추가할 함수] 마크다운 문법 무시하고 글자 크기 유지하는 함수
def escape_markdown_special_chars(text):
    if not text: return text
    # 1. 색상 코드(#RRGGBB)가 아닌 '#'만 찾아서 변환 (제목으로 변하는 것 방지)
    text = re.sub(r'#(?![0-9a-fA-F]{3,6})', '&#35;', text)
    # 2. 볼드체/기울임꼴(*, _) 방지
    text = text.replace('*', '&#42;').replace('_', '&#95;')
    return text

# --- 기존 함수들 (read_raw_docx, read_pdf, get_proofreading_chain) ---
def read_raw_docx(file_path):
    try:
        doc = docx.Document(file_path)
        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)
        return '\n'.join(full_text)
    except Exception as e:
        return f"파일을 읽는 중 오류가 발생했습니다: {str(e)}"

def read_pdf(file):
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def get_proofreading_chain(api_key):
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0, openai_api_key=api_key)
    template = """
    당신은 한국어 교정 전문가입니다. 아래 텍스트에서 오타, 비문, 어색한 표현을 찾아주세요.

    [텍스트]:
    {text}

    [응답 형식]:
    반드시 아래와 같은 **JSON 포맷**으로만 응답하세요. 다른 말은 하지 마세요.
    오류가 없으면 빈 리스트 [] 를 반환하세요.

    [
      {{
        "error_sentence": "오류가 포함된 원본 문장 또는 단어 구절 (원본 텍스트와 정확히 일치해야 함)",
        "correction": "수정 제안 내용",
        "reason": "수정 이유"
      }},
      ...
    ]
    """
    prompt = PromptTemplate.from_template(template)
    return prompt | llm | StrOutputParser()

# --- 메인 함수 ---
def main():

    st.set_page_config(page_title="Word/PPT 통합 플랫폼", layout='wide')

    if "proofreading_results" not in st.session_state:
        st.session_state.proofreading_results = None

    # [추가] 왼쪽 미리보기 화면용 상태 변수
    if "highlighted_preview" not in st.session_state:
        st.session_state.highlighted_preview = None

    st.title("문서 작업 통합 도구 (Correction & Conversion)")

    # [UI 구조 변경] 탭을 사용하여 기능 분리
    tab1, tab2 = st.tabs(["📄 문서 오타 검수", "🔄 PDF 일괄 변환"])

    # =========================================================
    # TAB 1: 기존 오타 검수 기능
    # =========================================================
    with tab1:
        col1, col2 = st.columns([1, 1])

        with col1:
            st.subheader("1. 파일 업로드 및 확인")
            openai_api_key = st.text_input("OpenAI API Key", type="password", key="api_key_tab1")
            uploaded_file = st.file_uploader("검수할 파일 업로드 (Word/PDF)", type=["docx", "pdf"], key="uploader_tab1")

            if uploaded_file is not None:
                # 임시 파일 처리 (기존 코드 유지)
                suffix = '.docx' if uploaded_file.name.endswith('.docx') else '.pdf'
                with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp_file:
                    tmp_file.write(uploaded_file.getvalue())
                    tmp_file_path = tmp_file.name

                # [수정] 미리보기 로직 변경
                # 원본 텍스트 읽기
                if suffix == '.docx':
                    raw_text = read_raw_docx(tmp_file_path)
                else:
                    raw_text = read_pdf(uploaded_file)

                # 검수 결과(하이라이팅)가 있으면 그것을 보여주고, 없으면 원본 텍스트를 보여줌
                if st.session_state.highlighted_preview:
                    st.markdown("⬇️ **오타가 감지된 문장이 빨갛게 표시됩니다.**")

                    preview_content = st.session_state.highlighted_preview

                    # CSS로 폰트 크기(14px)를 강제로 고정합니다.
                    st.markdown(
                        f"""
                        <div style="
                            height: 400px;
                            overflow-y: scroll;
                            border: 1px solid #dee2e6;
                            padding: 15px;
                            border-radius: 0.25rem;
                            background-color: #ffffff;
                            color: #31333F;
                            font-family: 'Apple SD Gothic Neo', 'Malgun Gothic', sans-serif;
                            font-size: 14px;
                            line-height: 1.6;">
                            {preview_content}
                        </div>
                        """,
                        unsafe_allow_html=True
                    )
                else:
                    st.text_area("내용 미리보기", raw_text, height=400)

        with col2:
            st.subheader("2. AI 검수 결과")
            if uploaded_file and suffix == '.docx':
                if st.button("오타 검수 시작", type="primary"):
                    if not openai_api_key:
                        st.error("API Key를 입력해주세요.")
                    else:
                        # ... (기존 초기화 코드) ...
                        sections = read_docx(tmp_file_path)
                        chain = get_proofreading_chain(openai_api_key)

                        results = []
                        full_highlighted_content = []  # [추가] 전체 문서를 재조립할 리스트

                        progress_bar = st.progress(0)

                        for i, section in enumerate(sections):
                            title = section.get('title', '제목 없음')
                            content = section.get('content', '')

                            try:
                                response_json = chain.invoke({"text": content})
                                # 하이라이팅 함수 호출
                                highlighted_text, errors = highlight_errors(content, response_json)

                                # 1. 글자가 커지는 것을 막기 위해 특수문자 처리 함수 통과
                                safe_highlighted = escape_markdown_special_chars(highlighted_text)

                                # 2. <h4> 태그 대신, 폰트 크기를 16px로 고정한 div 사용
                                section_html = f"""
                                <div style="margin-bottom: 20px;">
                                    <div style="font-size: 16px; font-weight: bold; color: #333; margin-bottom: 5px; border-bottom: 2px solid #eee; padding-bottom: 3px;">
                                        {title}
                                    </div>
                                    <div style="font-size: 14px; line-height: 1.6; color: #444;">
                                        {safe_highlighted}
                                    </div>
                                </div>
                                """
                                full_highlighted_content.append(section_html)

                            except Exception as e:
                                results.append({"title": title, "correction": f"에러: {e}"})
                                full_highlighted_content.append(f"<h4>{title}</h4>{content}<br>") # 에러 시 원본 유지

                            progress_bar.progress((i + 1) / len(sections))

                        st.session_state.proofreading_results = results

                        # [핵심] 분석이 끝나면 누적된 HTML을 세션 상태에 저장 -> 왼쪽 화면 갱신됨
                        st.session_state.highlighted_preview = "\n".join(full_highlighted_content)

                        progress_bar.empty()

                        # Streamlit 특성상 상태값 변경 후 즉시 UI 반영을 위해 rerun이 필요할 수 있음
                        st.rerun()

            # 결과 출력 루프 (app.py 하단부)
            if st.session_state.proofreading_results:
                st.markdown("---")
                st.subheader("📋 AI 수정 제안 요약")

                for res in st.session_state.proofreading_results:
                    # 각 섹션(문단 제목)별로 접고 펼칠 수 있는 영역 생성
                    with st.expander(f"📌 {res['title']}", expanded=True):

                        errors = res.get('errors', [])

                        # 1. 수정 사항이 없는 경우 깔끔하게 표시
                        if not errors:
                            st.markdown(
                                """
                                <div style='padding: 10px; background-color: #f0fdf4; color: #15803d; border-radius: 5px; border: 1px solid #bbf7d0;'>
                                    ✅ 발견된 오타나 수정 사항이 없습니다. 완벽합니다!
                                </div>
                                """,
                                unsafe_allow_html=True
                            )

                        # 2. 수정 사항이 있는 경우 (카드 형태로 나열)
                        else:
                            for idx, error in enumerate(errors):
                                # 데이터 가져오기 (없을 경우 빈칸 처리)
                                original_txt = error.get('error_sentence', '')
                                correction_txt = error.get('correction', '')
                                reason_txt = error.get('reason', '')

                                # 마크다운(#, *) 오작동 방지용 이스케이프 (이전 단계에서 만든 함수 활용 권장)
                                # 만약 함수가 없다면 이 부분은 생략 가능하지만, 안전을 위해 추천합니다.
                                # original_txt = escape_markdown_special_chars(original_txt)
                                # correction_txt = escape_markdown_special_chars(correction_txt)

                                # 스타일링된 HTML 카드 출력
                                st.markdown(f"""
                                <div style="
                                    background-color: white;
                                    border: 1px solid #e5e7eb;
                                    border-radius: 8px;
                                    padding: 15px;
                                    margin-bottom: 12px;
                                    box-shadow: 0 1px 2px rgba(0,0,0,0.05);
                                ">
                                    <div style="margin-bottom: 8px; font-size: 1.05em; line-height: 1.5;">
                                        <span style="background-color: #fee2e2; color: #991b1b; padding: 2px 6px; border-radius: 4px; font-weight: bold; text-decoration: line-through;">
                                            {original_txt}
                                        </span>
                                        <span style="margin: 0 8px; color: #6b7280;">➡️</span>
                                        <span style="background-color: #dcfce7; color: #166534; padding: 2px 6px; border-radius: 4px; font-weight: bold;">
                                            {correction_txt}
                                        </span>
                                    </div>

                                    <div style="
                                        background-color: #f9fafb;
                                        padding: 10px;
                                        border-radius: 6px;
                                        font-size: 0.9em;
                                        color: #4b5563;
                                        border-left: 3px solid #6366f1;
                                    ">
                                        <strong>💡 수정 이유:</strong> {reason_txt}
                                    </div>
                                </div>
                                """, unsafe_allow_html=True)
    # =========================================================
    # TAB 2: [신규] PDF 일괄 변환 기능
    # =========================================================
    with tab2:
        st.header("📂 Word/PPT -> PDF 일괄 변환")
        st.markdown("""
        - 지정한 폴더에 있는 **모든 .docx, .pptx 파일**을 PDF로 변환합니다.
        - **주의:** 서버(Linux)에 LibreOffice와 한글 폰트가 설치되어 있어야 정상 동작합니다.
        """)

        # 기본값을 현재 작업 경로로 설정
        default_path = os.getcwd()
        target_folder = st.text_input("변환할 파일이 있는 폴더 경로를 입력하세요:", value=default_path)

        if st.button("일괄 변환 시작", type="primary"):
            st.write("---")
            log_area = st.empty()

            # 제너레이터(yield)로부터 메시지를 받아 실시간 출력
            for msg_type, msg in batch_convert_to_pdf(target_folder):
                if msg_type == "Error":
                    st.error(msg)
                elif msg_type == "Success":
                    st.success(msg)
                elif msg_type == "Info":
                    st.info(msg)
                elif msg_type == "Progress":
                    with log_area:
                        st.write(f"⏳ {msg}")

            st.success("모든 작업이 종료되었습니다.")
            log_area.empty()

if __name__ == "__main__":
    main()

Overwriting app.py


In [25]:
import threading
import subprocess
import time
from pyngrok import ngrok

def run_stramlit():
  cmd = ["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"]
  subprocess.run(cmd)

thread = threading.Thread(target=run_stramlit, daemon=True)
thread.start()

time.sleep(5)

NGROK_AUTH_TOKEN = "31LySo6cMJ2KtrV8l3zVzZz0TGx_2TnffbAQuaW5vVffdPxf4"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(8501, "http")
print("외부 접속 URL : ", public_url)

외부 접속 URL :  NgrokTunnel: "https://ebd68ae7ed6c.ngrok-free.app" -> "http://localhost:8501"


In [23]:
!pgrep ngrok

18201


In [24]:
!kill 18201

In [ ]:
"""
[ { "error_sentence": "2. 연구 대상과 범위 한정 17", "correction": "2. 연구 대상 및 범위 17", "reason": "'범위 한정'이라는 표현은 어색하며, 목차 제목에서는 '연구 대상 및 범위'가 더 자연스럽고 일반적으로 쓰이는 표현입니다." }, { "error_sentence": "간추린 글(본문 대제목)’", "correction": "‘간추린 글(본문 대제목)’", "reason": "따옴표가 닫는 기호만 있어 쌍이 맞지 않습니다. 여는 따옴표를 추가하거나 양쪽 따옴표를 제거해야 합니다." }, { "error_sentence": "본문은 왼끝맞추기로 작성합니다.", "correction": "본문은 왼쪽 정렬로 작성합니다.", "reason": "'왼끝맞추기'는 표준어가 아니며 어색한 표현입니다. 편집 용어로는 '왼쪽 정렬' 또는 '왼쪽 맞춤'이 적절합니다." }, { "error_sentence": "‘용어 제목2’ 목록으로 작성합니다.(용어제목2)", "correction": "‘용어 제목2’ 목록으로 작성합니다. (용어제목2)", "reason": "마침표와 괄호 사이에 공백이 없어 가독성이 떨어집니다. 문장부호 뒤에는 공백을 넣는 것이 일반적입니다." }, { "error_sentence": "‘본문 대제목’ 목록으로 작성합니다.(본문 대제목 1)", "correction": "‘본문 대제목’ 목록으로 작성합니다. (본문 대제목 1)", "reason": "마침표와 괄호 사이에 공백이 없어 가독성이 떨어집니다. 문장부호 뒤에 공백을 추가하는 것이 좋습니다." }, { "error_sentence": "‘목차 제목1’의 스타일을 적용합니다", "correction": "‘목차 제목1’의 스타일을 적용합니다.", "reason": "동일한 유형의 문장에 마침표 사용이 혼재되어 있습니다. 문장 끝에 마침표를 일관되게 사용해야 합니다." } ]
"""